# 🚀 Story Video Generator - GPU Server (Kokoro TTS + SDXL-Turbo + FFmpeg)

**Requirements**: 
- Runtime: GPU (T4, V100, A100)
- GPU RAM: 15+ GB

**Features**:
- 🎤 Kokoro TTS (8 voices)
- 🎨 SDXL-Turbo (AI image generation)
- 🎬 FFmpeg (video processing)
- ⚡ On-demand model loading (memory optimized)
- 🌐 Ngrok public URL
- 📡 3 endpoints: /generate_audio, /generate_image, /generate_images_batch

---

## ⚠️ IMPORTANT: Enable GPU Runtime

**Before running, make sure GPU is enabled:**
1. Click: `Runtime` → `Change runtime type`
2. Select: `Hardware accelerator` → `GPU` → `T4 GPU`
3. Click: `Save`
4. Run all cells

---

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 📦 STEP 1: INSTALL DEPENDENCIES
# ═══════════════════════════════════════════════════════════════════════════════

print("📦 Installing dependencies...\n")

# ✅ FFmpeg (for video processing)
print("🎬 Installing FFmpeg...")
!apt-get update -qq
!apt-get install -y -qq ffmpeg
!ffmpeg -version | head -n 1
print("   ✅ FFmpeg installed!\n")

# Core dependencies
!pip install -q flask flask-cors pyngrok

# Kokoro TTS - ✅ FIXED: Install from GitHub source (latest working version)
!pip install -q git+https://github.com/thewh1teagle/kokoro-onnx.git
!pip install -q soundfile numpy scipy onnxruntime

# SDXL-Turbo (Diffusers)
!pip install -q diffusers transformers accelerate safetensors

# Torch (GPU support)
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

print("\n✅ All dependencies installed!")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🔧 STEP 2: SETUP GPU & IMPORTS
# ═══════════════════════════════════════════════════════════════════════════════

# ✅ Safety check: Ensure Cell 1 dependencies are installed
try:
    from flask_cors import CORS
except ImportError:
    print("⚠️  Dependencies not found! Running Cell 1 installations first...\n")
    # Install core dependencies if missing
    import sys
    !{sys.executable} -m pip install -q flask flask-cors pyngrok
    print("✅ Core dependencies installed!\n")

# Now import everything
import os
import gc
import torch
import json
from flask import Flask, request, jsonify, send_file
from flask_cors import CORS
from pyngrok import ngrok
from pathlib import Path
from threading import Thread

# GPU Detection
if torch.cuda.is_available():
    device = 'cuda'
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"✅ GPU ENABLED: {gpu_name}")
    print(f"💾 GPU Memory: {gpu_memory:.1f} GB")
    print(f"🔥 CUDA Version: {torch.version.cuda}")
else:
    device = 'cpu'
    print("⚠️  WARNING: GPU NOT DETECTED - Running on CPU (SLOW!)")
    print("⚠️  Please enable GPU: Runtime → Change runtime type → GPU")

print(f"\n🚀 Device: {device}")

# Create output directory
output_dir = Path('/content/outputs')
output_dir.mkdir(exist_ok=True)

print(f"📁 Output directory: {output_dir}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🧠 STEP 3: MEMORY MANAGEMENT (On-Demand Model Loading)
# ═══════════════════════════════════════════════════════════════════════════════

# Global model holders (loaded on-demand)
tts_pipeline = None
img_pipeline = None

def clear_gpu_memory():
    """Clear GPU memory to make room for other models"""
    global tts_pipeline, img_pipeline
    
    if tts_pipeline is not None:
        del tts_pipeline
        tts_pipeline = None
        print("   🗑️  Unloaded TTS model")
    
    if img_pipeline is not None:
        del img_pipeline
        img_pipeline = None
        print("   🗑️  Unloaded Image model")
    
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()

def load_tts_model():
    """Load Kokoro TTS model (unload image model first)"""
    global tts_pipeline, img_pipeline
    
    if tts_pipeline is not None:
        return tts_pipeline
    
    print("\n🎤 Loading Kokoro TTS...")
    
    # Unload image model if loaded
    if img_pipeline is not None:
        print("   ⚠️  Unloading image model to free memory...")
        clear_gpu_memory()
    
    # ✅ FIXED: Direct import (kokoro already installed in Cell 1)
    # No try/except - if this fails, Cell 1 needs to be re-run
    from kokoro import KPipeline
    
    # Initialize with American English (code 'a')
    tts_pipeline = KPipeline(lang_code='a')
    if device == 'cuda' and torch.cuda.is_available():
        tts_pipeline = tts_pipeline.to('cuda')
        print("   ✅ Kokoro TTS loaded on GPU!")
    else:
        print("   ✅ Kokoro TTS loaded on CPU!")
    
    return tts_pipeline

def load_image_model():
    """Load SDXL-Turbo model (unload TTS model first)"""
    global tts_pipeline, img_pipeline
    
    if img_pipeline is not None:
        return img_pipeline
    
    print("\n🎨 Loading SDXL-Turbo...")
    
    # Unload TTS model if loaded
    if tts_pipeline is not None:
        print("   ⚠️  Unloading TTS model to free memory...")
        clear_gpu_memory()
    
    # ✅ FIXED: Use DiffusionPipeline (works with all diffusers versions)
    from diffusers import DiffusionPipeline
    import torch
    
    # Load model with correct dtype based on device
    img_pipeline = DiffusionPipeline.from_pretrained(
        "stabilityai/sdxl-turbo",
        torch_dtype=torch.float16 if device == 'cuda' else torch.float32,
        variant="fp16" if device == 'cuda' else None,
        use_safetensors=True
    )
    
    # Move to GPU and optimize
    if device == 'cuda':
        img_pipeline = img_pipeline.to(device)
        # Memory optimization for T4 GPU (15GB)
        img_pipeline.enable_attention_slicing()
        img_pipeline.enable_vae_slicing()
        print("   ⚡ Memory optimization enabled")
    
    print("   ✅ SDXL-Turbo loaded!")
    return img_pipeline

print("✅ Memory management functions ready!")
print("   📌 Models load on-demand to save memory")
print("   📌 Automatic unloading when switching models")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🎤 STEP 4: VOICE MAPPING (Kokoro TTS)
# ═══════════════════════════════════════════════════════════════════════════════

# Voice mapping: Backend voice names → Kokoro voice codes
VOICE_MAPPING = {
    # Male voices (backend → Kokoro)
    'guy': 'af_adam',
    'adam_narration': 'af_adam',
    'adam_business': 'af_adam',
    'michael': 'af_michael',
    'george_gb': 'af_michael',
    'brian': 'af_adam',
    'andrew': 'af_adam',
    'christopher': 'af_michael',
    'george': 'af_michael',
    'joey': 'af_adam',
    
    # Female voices (backend → Kokoro)
    'aria': 'af_bella',
    'sarah_pro': 'af_sarah',
    'sarah_natural': 'af_sarah',
    'nicole': 'af_nicole',
    'emma_gb': 'af_bella',
    'jenny': 'af_nicole',
    'sara': 'af_sarah',
    'jane': 'af_nicole',
    'libby': 'af_bella',
    'emma': 'af_bella',
    'ivy': 'af_bella'
}

print("✅ Voice mapping configured:")
print("   Backend voices → Kokoro voices")
for key, value in list(VOICE_MAPPING.items())[:8]:
    print(f"   {key:20} → {value}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🌐 STEP 5: FLASK API SERVER
# ═══════════════════════════════════════════════════════════════════════════════

app = Flask(__name__)
CORS(app)

@app.route('/health', methods=['GET'])
def health():
    """Health check endpoint"""
    return jsonify({
        'status': 'healthy',
        'device': device,
        'gpu': torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None',
        'models_loaded': {
            'tts': tts_pipeline is not None,
            'image': img_pipeline is not None
        }
    })

@app.route('/generate_audio', methods=['POST'])
def generate_audio():
    """Generate audio using Kokoro TTS"""
    try:
        data = request.json
        text = data.get('text', '')
        voice = data.get('voice', 'guy')
        speed = float(data.get('speed', 1.0))
        
        if not text:
            return jsonify({'error': 'No text provided'}), 400
        
        # Map voice to Kokoro voice code
        kokoro_voice = VOICE_MAPPING.get(voice, 'af_adam')
        
        print(f"\n🎤 Generating audio: {voice} → {kokoro_voice}")
        print(f"   Text: {text[:50]}...")
        
        # Load TTS model
        pipeline = load_tts_model()
        
        # Generate audio - ✅ FIXED: Use correct KPipeline API
        audio_path = output_dir / f"audio_{hash(text)}.wav"
        
        # Generate audio with KPipeline (returns generator)
        import soundfile as sf
        import numpy as np
        
        audio_segments = []
        for i, (gs, ps, audio) in enumerate(pipeline(text, voice=kokoro_voice, speed=speed)):
            audio_segments.append(audio)
        
        # Concatenate all segments
        if len(audio_segments) > 0:
            full_audio = np.concatenate(audio_segments)
            sample_rate = 24000  # Kokoro's sample rate
            
            # Save audio to file
            sf.write(str(audio_path), full_audio, sample_rate)
        else:
            raise RuntimeError("No audio generated")
        
        print(f"   ✅ Audio generated: {audio_path.name}")
        
        return send_file(
            audio_path,
            mimetype='audio/wav',
            as_attachment=True,
            download_name=audio_path.name
        )
    
    except Exception as e:
        print(f"   ❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return jsonify({'error': str(e)}), 500

@app.route('/generate_image', methods=['POST'])
def generate_image():
    """Generate a single image using SDXL-Turbo"""
    try:
        data = request.json
        prompt = data.get('prompt', '')
        style = data.get('style', 'cinematic')
        
        if not prompt:
            return jsonify({'error': 'No prompt provided'}), 400
        
        # Add style to prompt
        full_prompt = f"{prompt}, {style} style, high quality, detailed"
        
        print(f"\n🎨 Generating image (16:9)...")
        print(f"   Prompt: {full_prompt[:70]}...")
        
        # Load image model
        pipeline = load_image_model()
        
        # ✅ FIXED: Clear cache before generation
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        
        # ✅ FIXED: Generate 16:9 ratio (1920x1080) for TikTok/YouTube
        with torch.inference_mode():
            image = pipeline(
                prompt=full_prompt,
                num_inference_steps=4,  # SDXL-Turbo optimized for 1-4 steps
                guidance_scale=0.0,      # SDXL-Turbo doesn't use guidance
                height=1080,             # ✅ 16:9 aspect ratio (Full HD)
                width=1920
            ).images[0]
        
        # Save image
        image_path = output_dir / f"image_{hash(prompt)}.png"
        image.save(image_path, format='PNG', optimize=True)
        
        print(f"   ✅ Image generated (1920x1080): {image_path.name}")
        
        # Clear GPU cache
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        
        return send_file(
            image_path,
            mimetype='image/png',
            as_attachment=True,
            download_name=image_path.name
        )
    
    except Exception as e:
        print(f"   ❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return jsonify({'error': str(e)}), 500

@app.route('/generate_images_batch', methods=['POST'])
def generate_images_batch():
    """✅ Generate multiple images in 16:9 ratio and return as base64"""
    try:
        data = request.json
        scenes = data.get('scenes', [])
        style = data.get('style', 'cinematic')
        
        if not scenes:
            return jsonify({'error': 'No scenes provided'}), 400
        
        print(f"\n🎨 Generating {len(scenes)} images (16:9 ratio)...")
        
        # Load image model once
        pipeline = load_image_model()
        
        results = []
        
        for i, scene in enumerate(scenes, 1):
            prompt = scene.get('description', '')
            if not prompt:
                results.append({
                    'success': False,
                    'error': 'No prompt',
                    'scene_index': i - 1
                })
                continue
            
            full_prompt = f"{prompt}, {style} style, high quality, detailed"
            
            print(f"   [{i}/{len(scenes)}] {prompt[:50]}...")
            
            try:
                # ✅ Clear cache BEFORE each generation to prevent OOM
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
                
                # ✅ Generate 16:9 ratio (1920x1080) images
                with torch.inference_mode():
                    image = pipeline(
                        prompt=full_prompt,
                        num_inference_steps=4,
                        guidance_scale=0.0,
                        height=1080,  # ✅ 16:9 aspect ratio (Full HD)
                        width=1920
                    ).images[0]
                
                # ✅ Convert image to base64 for transfer
                import io
                import base64
                
                buffer = io.BytesIO()
                image.save(buffer, format='PNG')
                image_bytes = buffer.getvalue()
                image_base64 = base64.b64encode(image_bytes).decode('utf-8')
                
                results.append({
                    'success': True,
                    'image_data': image_base64,  # ✅ Base64 encoded PNG
                    'scene_index': i - 1,
                    'size_bytes': len(image_bytes),
                    'resolution': '1920x1080'  # ✅ 16:9 ratio
                })
                
                print(f"      ✅ Generated: 1920x1080, {len(image_bytes):,} bytes")
            
            except torch.cuda.OutOfMemoryError as e:
                print(f"      ❌ CUDA OOM Error: {e}")
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
                results.append({
                    'success': False,
                    'error': 'CUDA out of memory',
                    'scene_index': i - 1
                })
            
            except Exception as e:
                print(f"      ❌ Error: {e}")
                import traceback
                traceback.print_exc()
                results.append({
                    'success': False,
                    'error': str(e),
                    'scene_index': i - 1
                })
        
        success_count = len([r for r in results if r.get('success')])
        total_size = sum(r.get('size_bytes', 0) for r in results if r.get('success'))
        
        print(f"   ✅ Batch complete: {success_count}/{len(scenes)} successful")
        print(f"   📦 Total data: {total_size:,} bytes ({total_size/1024/1024:.1f} MB)")
        print(f"   📐 Resolution: 1920x1080 (16:9 aspect ratio)")
        
        return jsonify({'results': results})
    
    except Exception as e:
        print(f"   ❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return jsonify({'error': str(e)}), 500

print("\n✅ Flask API configured with 4 endpoints:")
print("   /health               - Health check")
print("   /generate_audio       - Kokoro TTS (single)")
print("   /generate_image       - SDXL-Turbo (single, 16:9)")
print("   /generate_images_batch - SDXL-Turbo (batch, 16:9, base64)")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🌐 STEP 6: NGROK SETUP (Public URL)
# ═══════════════════════════════════════════════════════════════════════════════

print("\n🔑 Setting up Ngrok...")

# Your Ngrok auth token (already configured!)
NGROK_AUTH_TOKEN = "35HuufK0IT26RER84mcvIbRjrog_7grjZvuDXtRPYL5hWLNCK"

ngrok.set_auth_token(NGROK_AUTH_TOKEN)
print("✅ Ngrok token configured!")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🚀 STEP 7: START SERVER
# ═══════════════════════════════════════════════════════════════════════════════

def run_server():
    """Run Flask server in background thread"""
    # ✅ FIXED: Use port 5001 to avoid conflict with local backend
    app.run(host='0.0.0.0', port=5001, debug=False, use_reloader=False)

# Start Flask in background
print("\n🚀 Starting Flask server...")
server_thread = Thread(target=run_server, daemon=True)
server_thread.start()

# Wait for server to start
import time
time.sleep(3)

# Start Ngrok tunnel - ✅ FIXED: Point to port 5001
print("🌐 Starting Ngrok tunnel...")
public_url = ngrok.connect(5001, bind_tls=True)

print("\n" + "═" * 80)
print("🎉 SERVER RUNNING!")
print("═" * 80)
print(f"\n📡 Public URL: {public_url.public_url}")
print(f"🖥️  Local URL:  http://localhost:5001")
print(f"\n🎮 Device: {device.upper()}")
if torch.cuda.is_available():
    print(f"🔥 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

print("\n📌 API Endpoints:")
print(f"   {public_url.public_url}/health")
print(f"   {public_url.public_url}/generate_audio")
print(f"   {public_url.public_url}/generate_image")
print(f"   {public_url.public_url}/generate_images_batch")

print("\n" + "═" * 80)
print("🔧 UPDATE YOUR BACKEND CONFIG:")
print("═" * 80)
print(f"\n   File: story-video-generator/config/__init__.py")
print(f"   Line 16: COLAB_SERVER_URL = '{public_url.public_url}'")
print("\n" + "═" * 80)

print("\n💡 Server will run until you stop this cell or disconnect from Colab")
print("═" * 80)

# Keep server running
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("\n🛑 Server stopped!")